# Install packages

In [1]:
!pip install ibm-ai-openscale --no-cache 

     |████████████████████████████████| 491kB 17.3MB/s eta 0:00:01


# Import required packages

In [2]:
import json
import random

# Configure Credentials & Parameters

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

The Subscription ID is visible on OpenScale -> Configure Monitors

In [3]:
CLOUD_API_KEY = "xxx"
SUBSCRIPTION_ID = "xxx"
FEEDBACK_THRESHOLD = 50

In [4]:
# The code was removed by Watson Studio for sharing.

# Configure OpenScale

The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [5]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

### Get Watson OpenScale GUID

Each instance of OpenScale has a unique ID. We can get this value using the Cloud API key specified at the beginning of the notebook.

In [6]:
import requests

AIOS_GUID = None
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': CLOUD_API_KEY
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

resources = json.loads(requests.get('https://resource-controller.cloud.ibm.com/v2/resource_instances', headers=iam_headers).text)['resources']
for resource in resources:
    if "aiopenscale" in resource['id'].lower():
        AIOS_GUID = resource['guid']
        
AIOS_CREDENTIALS = {
    "instance_guid": AIOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

if AIOS_GUID is None:
    print('Watson OpenScale GUID NOT FOUND')
else:
    print(AIOS_GUID)

0379da8c-654a-4874-b7aa-a1d030d600d6


In [7]:
ai_client = APIClient(aios_credentials=AIOS_CREDENTIALS)
ai_client.version

'2.1.11'

In [8]:
subscription = ai_client.data_mart.subscriptions.get(SUBSCRIPTION_ID)

# Feedback logging

### Download feedback data

The code below downloads and stores enough feedback data to meet the minimum threshold so that OpenScale can calculate a new accuracy measurement. It then kicks off the accuracy monitor. The monitors run hourly, or can be initiated via the Python API, the REST API, or the graphical user interface.

In [9]:
!rm additional_feedback_data.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/additional_feedback_data.json

rm: cannot remove ‘additional_feedback_data.json’: No such file or directory
--2019-07-22 10:46:41--  https://raw.githubusercontent.com/emartensibm/german-credit/master/additional_feedback_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16506 (16K) [text/plain]
Saving to: ‘additional_feedback_data.json’

100%[======================================>] 16,506      --.-K/s   in 0.001s  

2019-07-22 10:46:41 (11.1 MB/s) - ‘additional_feedback_data.json’ saved [16506/16506]



In [10]:
with open('additional_feedback_data.json') as feedback_file:
    additional_feedback_data = json.load(feedback_file)

### Create a random sample

... just big enough to meet the feedback threshold

In [11]:
randomsample = random.sample(additional_feedback_data['data'], FEEDBACK_THRESHOLD)

### Upload the feedback data

In [12]:
subscription.feedback_logging.store(randomsample)

In [13]:
subscription.feedback_logging.describe_table()

       LoanDuration    LoanAmount  InstallmentPercent  \
count    199.000000    199.000000          199.000000   
mean      20.000000   3391.793970            2.874372   
std       11.503623   2762.543767            1.077453   
min        4.000000    250.000000            1.000000   
25%       10.000000   1210.500000            2.000000   
50%       20.000000   2963.000000            3.000000   
75%       28.000000   5098.000000            4.000000   
max       52.000000  10584.000000            5.000000   

       CurrentResidenceDuration         Age  ExistingCreditsCount  Dependents  
count                199.000000  199.000000            199.000000  199.000000  
mean                   2.713568   35.080402              1.407035    1.170854  
std                    1.203264   10.083669              0.550620    0.377331  
min                    1.000000   19.000000              1.000000    1.000000  
25%                    2.000000   28.000000              1.000000    1.000000  
50%   